In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

import joblib

# 필요한 라이브러리, 함수 로딩 ------------------

from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
# 하이퍼 파라미터 튜닝은 _val로 진행

In [37]:
# 이찬울님 의견  data_test2.csv, data_train2.csv 사용 : 컬럼명들을 v1~561로 변환 
data_train2 = pd.read_csv('data01_train2.csv')
data_test2= pd.read_csv('data01_test2.csv')

# train data 전처리
drop_cols = 'subject'
data_test2.drop(columns = drop_cols ,inplace =True)
data_train2.drop(columns = drop_cols, inplace = True)

# 학슴용 검증용 데이터 분할
target = 'Activity'
x = data_train2.drop(columns= target)
y = data_train2.loc[:,target]

x_train , x_val , y_train , y_val = train_test_split(x,y,random_state=42,test_size=0.2)

# 평가용 데이터 분할
x_test = data_test2.drop(columns= target)
y_test = data_test2.loc[:,target]

# 라벨링
activity_mapping = {
    'LAYING': 0,
    'STANDING': 1,
    'SITTING': 2,
    'WALKING': 3,
    'WALKING_UPSTAIRS': 4,
    'WALKING_DOWNSTAIRS': 5
}
y_train = y_train.map(activity_mapping)
y_val = y_val.map(activity_mapping)
y_test = y_test.map(activity_mapping)


# 학습용 데이터 스케일링 학습 
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train) # 학습용 데이터는 fit 학습을 수행하고 변환하지만
x_val= scaler.transform(x_val)# 검증용 데이터는 fit 을 수행해선 안됀다 > 평가용 데이터 역시 fit을 해선 안됀다 
x_test= scaler.transform(x_test)

# 모델 생성 학습

model_lgbm = LGBMClassifier(n_estimators=260 ,learning_rate = 0.422,verbose =-1)# 최적의 파리미터는 이미 생성했습니다 
model_lgbm.fit(x_train,y_train)

y_pred = model_lgbm.predict(x_val)

# 정확도 평가(검증용)
print('============검증 결과==================')
print(confusion_matrix(y_val, y_pred))
print('Classification Report  \n' , classification_report(y_val, y_pred ))

y_pred = model_lgbm.predict(x_test)

# 정확도 평가(평가용)
print('============평가 결과==================')
print(confusion_matrix(y_test, y_pred))
print('Classification Report  \n' , classification_report(y_test, y_pred ))
 

============검증 결과==================
[[231   0   0   0   0   0]
 [  0 222   4   0   0   0]
 [  0   1 199   0   0   0]
 [  0   0   0 197   1   0]
 [  0   0   0   0 177   0]
 [  0   0   0   0   0 145]]
Classification Report  
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       231
           1       1.00      0.98      0.99       226
           2       0.98      0.99      0.99       200
           3       1.00      0.99      1.00       198
           4       0.99      1.00      1.00       177
           5       1.00      1.00      1.00       145

    accuracy                           0.99      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       0.99      0.99      0.99      1177

============평가 결과==================
[[291   0   0   0   0   1]
 [  0 281   6   0   0   0]
 [  0   5 249   0   0   0]
 [  0   0   0 227   1   0]
 [  0   0   0   0 215   0]
 [  0   0   0   1   0 194]]
Classification Report  
          